In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code A6C2HZM7Y to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-139656
Azure region: southcentralus
Subscription id: a24a24d5-8d87-4c8a-99b6-91ed2d2df51f
Resource group: aml-quickstarts-139656


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cpu_cluster_name = 'compute-cluster'

# very the cluster is not aleady created
if cpu_cluster_name in ws.compute_targets:
    compute_target = ws.compute_targets[cpu_cluster_name]
    print('Found an existing cluster to use')
else:
    print('Creating a new compute cluster')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

print(compute_target.get_status().serialize())

Creating a new compute cluster
Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-02-28T23:59:39.047000+00:00', 'errors': None, 'creationTime': '2021-02-28T23:59:36.844602+00:00', 'modifiedTime': '2021-02-28T23:59:52.827634+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.core import ScriptRunConfig
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C': choice(0.5, 1),
        '--max_iter': choice(50, 100, 150)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(
    source_directory=os.path.join('./'),
    entry_script='train.py',
    compute_target=compute_target)

# est = ScriptRunConfig(
#     source_directory=os.path.join('./'),
#     script='train.py',
#     compute_target=compute_target
# )

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    estimator=est,
    hyperparameter_sampling=ps,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    policy=policy,
    max_total_runs=20,
    max_concurrent_runs=4
)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(hyperdrive_config)

hyperdrive_run.wait_for_completion(show_output=True)

assert(hyperdrive_run.get_status() == 'Completed')

RunId: HD_258e06e3-b5a6-439c-99cf-5681eb5a1909
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_258e06e3-b5a6-439c-99cf-5681eb5a1909?wsid=/subscriptions/a24a24d5-8d87-4c8a-99b6-91ed2d2df51f/resourcegroups/aml-quickstarts-139656/workspaces/quick-starts-ws-139656

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-03-01T00:00:54.834564][API][INFO]Experiment created<END>\n""<START>[2021-03-01T00:00:55.512179][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-03-01T00:00:55.689784][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2021-03-01T00:00:56.7498845Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_258e06e3-b5a6-439c-99cf-5681eb5a1909
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_258e06e3-b5a6-439c-99cf-5681eb5a1909?wsid=/subscriptions/a24a2

In [5]:
from azureml.widgets import RunDetails

RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [6]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
print(hyperdrive_run.get_children_sorted_by_primary_metric(top=0, reverse=False, discard_no_metric=False))

best_model = hyperdrive_run.get_best_run_by_primary_metric()

print('Best Model:', best_model.get_metrics())
print('Best Model Details:', best_model.get_details())
print('Best Model File Name:', best_model.get_file_names())

[{'run_id': 'HD_dee52e62-96b1-48c9-b208-4f746bec96d2_4', 'hyperparameters': None, 'best_primary_metric': 0.9118866970156804, 'status': 'Completed'}, {'run_id': 'HD_dee52e62-96b1-48c9-b208-4f746bec96d2_3', 'hyperparameters': None, 'best_primary_metric': 0.9118866970156804, 'status': 'Completed'}, {'run_id': 'HD_dee52e62-96b1-48c9-b208-4f746bec96d2_0', 'hyperparameters': None, 'best_primary_metric': 0.9118866970156804, 'status': 'Completed'}, {'run_id': 'HD_dee52e62-96b1-48c9-b208-4f746bec96d2_5', 'hyperparameters': None, 'best_primary_metric': 0.9109762266059687, 'status': 'Completed'}, {'run_id': 'HD_dee52e62-96b1-48c9-b208-4f746bec96d2_2', 'hyperparameters': None, 'best_primary_metric': 0.9109762266059687, 'status': 'Completed'}, {'run_id': 'HD_dee52e62-96b1-48c9-b208-4f746bec96d2_1', 'hyperparameters': None, 'best_primary_metric': 0.9109762266059687, 'status': 'Completed'}, {'run_id': 'HD_dee52e62-96b1-48c9-b208-4f746bec96d2_preparation', 'hyperparameters': None, 'best_primary_metric

In [7]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files(["https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"])

In [8]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [9]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    n_cross_validations=3)

In [10]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(automl_config, show_output=True)
automl_run.wait_for_completion()

No run_configuration provided, running on local with default configuration
Running on local machine
Parent Run ID: AutoML_ac15a2bc-9135-4a8e-b32c-41bc227d2386

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/Auto

{'runId': 'AutoML_ac15a2bc-9135-4a8e-b32c-41bc227d2386',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2021-02-21T23:20:37.970432Z',
 'endTimeUtc': '2021-02-21T23:54:01.590777Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"9e65f93e-bdd8-437b-b1e8-0647cd6098f7","resource_group":"aml-quickstarts-139297","workspace_name":"quick-starts-ws-139297","region":"southcentralus","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":3,"y_min":null,"y_max":null,"num_classes":2,"f

In [11]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
automl_best_model, fitted_model = automl_run.get_output()

print('Best Model:', automl_best_model.get_metrics())
print('Best Model Details:', automl_best_model.get_details())
print('Best Model File Name:', automl_best_model.get_file_names())

print(automl_best_model)

Best Model: {'average_precision_score_weighted': 0.9565486914800393, 'AUC_weighted': 0.9489314308666811, 'precision_score_micro': 0.9171471345384313, 'accuracy': 0.9171471345384313, 'matthews_correlation': 0.5494298123650826, 'AUC_micro': 0.9811331384549521, 'balanced_accuracy': 0.7493149312839306, 'weighted_accuracy': 0.9588637581099476, 'average_precision_score_macro': 0.8288644689486205, 'norm_macro_recall': 0.49862986256786107, 'recall_score_micro': 0.9171471345384313, 'precision_score_weighted': 0.9112425687198118, 'f1_score_micro': 0.9171471345384313, 'recall_score_weighted': 0.9171471345384313, 'f1_score_weighted': 0.9131832573403118, 'AUC_macro': 0.9489314308666811, 'log_loss': 0.1733239441352216, 'recall_score_macro': 0.7493149312839306, 'average_precision_score_micro': 0.9819419683314194, 'f1_score_macro': 0.7721859651519964, 'precision_score_macro': 0.8029735069623145, 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_ac15a2bc-9135-4a8e-b32c-41bc227d2386_42/con

In [6]:
compute_target.delete()

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

